<a href="https://colab.research.google.com/github/Eduardo7660/Organiza-o-M2A-e-M2B/blob/main/TrabalhoArq2_M2A_M2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output

Classes Utilizadas

In [ ]:
class LinhaASM:
    """
   Representa uma instrução de asembly.
    """

    def __init__(self):
      """
     Inicializa um objeto LinhaASM vazio.
      """
      #no ruby n precida disso :v
      self.instrucao = ""  # Complete instruction (32 bits)
      self.opcode = ""      # Opcode substring
      self.rd = ""          # rd register substring
      self.funct3 = ""       # funct3 substring
      self.rs1 = ""          # rs1 register substring
      self.rs2 = ""          # rs2 register substring
      self.funct7 = ""       # funct7 substring
      self.tipoInstrucao = ""  # Instruction type
      self.is_manual_nop = False  # Flag for manual NOP


In [ ]:
def contar_nops(programa):
    nops = sum(1 for linha in programa if linha.is_manual_nop)
    print(f"TOTAL DE NOPs inseridos: {nops}")

In [ ]:
class Organizacao:
  """
Representa a organização do processador.
  """

  def __init__(self):
    """
    Inicializa a organização com valores padrão.
  """
    self.TClock = 1.0  # Clock period
    self.freqClock = 1.0 / self.TClock  # Clock frequency (1/TClock)
    self.quantCiclos = {
        "U": 1.0,
        "J": 1.0,
        "B": 1.0,
        "I_ar": 1.0,  # Cycles for arithmetic immediates and ecall
        "I_lo": 1.0,  # Cycles for load immediates
        "R": 1.0,
        "S": 1.0
    }

In [ ]:
class Resultados:
  """
 Representa os resultados de desempenho do processador.
  """

  def __init__(self, ciclos_totais = 0, cpi = 0, tempo_exec = 0, desempenho = 0):
    """
   Inicializa o objeto Resultados com os valores especificados.

    Argumentos:
      ciclos_totais: Total de ciclos de CPU.
      cpi: Ciclos por instrução.
      tempo_exec: tempo de execução da CPU em segundos.
      desempenho: Desempenho em instruções por segundo.
    """
    self.ciclos_totais = ciclos_totais
    self.cpi = cpi
    self.tempo_exec = tempo_exec
    self.desempenho = desempenho

Funções de calculos

In [ ]:
def gerar_desempenho(tempo_execucao):
  """
 Calcula o desempenho (instruções por segundo) com base no tempo de execução.

  Argumento:
    tempo_execucao: Tempo de execução em segundos.

  Retorna:
    Desempenho em instruções por segundo.
  """
  return 1.0 / tempo_execucao

def t_exec_cpu_por_tempo_clock(quant_instrucoes, cpi, tempo_clock):
    return quant_instrucoes * cpi * tempo_clock

def gerar_cpi(ciclos_cpu, quant_instrucoes):
  """
 Calcula os ciclos por instrução (CPI) com base no total de ciclos da CPU e na contagem de instruções.

  Argumentos:
    ciclos_cpu: Total de ciclos de CPU.
    quant_instrucoes: Número de instruções.

  Retorna:
    Ciclos por instrução.
  """
  return (5.0 + (1.0 * (quant_instrucoes - 1))) / quant_instrucoes


Funções

In [ ]:
def verificar_hazards(instrucoes, forwarding_implementado):
  """
  Harzard = PERIGO PERIGO
  """

  print("Executando verificação de hazards")
  hazards = []  #Lista para armazenar índices de instruções com perigos

  for i in range(len(instrucoes)):
    # ignora a instrução se o registrador de destino é o zero ou é um nop
    if instrucoes[i].tipo_instrucao != "S" and instrucoes[i].rd == "00000":
      continue

    # Verifique se há perigos duas instruções antes de i
    for j in range(i + 1, i + 3):
      # Ignore a iteração j se ela exceder o número de instruções
      if j >= len(instrucoes):
        continue

      #Ignore a segunda iteração se o encaminhamento for implementado
      if j == i + 2 and forwarding_implementado:
        continue

      # Verifique se há perigo entre as instruções i e j
      if verificar_hazard_instrucao(instrucoes[i], instrucoes[j], forwarding_implementado):
        print(f"| Hazard encontrado  {j + 1} da linha inacabada {i + 1}")
        hazards.append(i)

  if not hazards:
    print("Nenhum hazards encontrado")

  print("Verificação de hazard concluída com sucesso")
  return hazards


In [ ]:
def aplicar_reordenacao(instrucoes, hazards, forwarding_implementado):
  """
  Aplica reordenação de instruções para eliminar hazard.

  Argumentos:
    instrucoes: Uma lista de objetos LinhaASM que representam as instruções.
    hazard: Uma lista de índices de hazard.
    forwarding_implementado: Um booleano que indica se o encaminhamento está implementado.

  Retorna:
    A lista modificada de objetos LinhaASM.
  """

  # Iterar por todos os hazard
  for i in range(len(hazards)):
    linha_escolhida = None
    linha_escolhida_definida = False
    indice_linha_escolhida = 0

    # Iterar pelas instruções após o hazard
    for j in range(i + 1, len(instrucoes)):
      # Valida a instrução e move para depois do hazard
      if verificar_hazard_instrucao(instrucoes[hazards[i]], instrucoes[j], forwarding_implementado):
        continue

      # Verifique se há conflitos sem encaminhar para hazard anteriores
      if not forwarding_implementado and i > 0:
        if verificar_hazard_instrucao(instrucoes[hazards[i] - 1], instrucoes[j], forwarding_implementado):
          continue

      # Verifique se há conflitos com instruções após a instrução movida
      linha_valida_depois = True
      for k in range(hazards[i] + 1, hazards[i] + (2 if forwarding_implementado else 1)):
        if k >= len(instrucoes):
          continue

        if verificar_hazard_instrucao(instrucoes[k], instrucoes[j], forwarding_implementado):
          linha_valida_depois = False
          break

      # Valide a instrução antes de movê-la
      linha_valida_antes = True
      for k in range(j + 1, j + (2 if forwarding_implementado else 1)):
        if k >= len(instrucoes):
          continue

        if verificar_hazard_instrucao(instrucoes[j - 1], instrucoes[k], forwarding_implementado):
          linha_valida_antes = False
          break

      # Se as verificações antes e depois passarem, podemos mover a instrução
      if linha_valida_antes and linha_valida_depois:
        linha_escolhida = instrucoes[j]
        linha_escolhida_definida = True
        indice_linha_escolhida = j

    # Se uma instrução adequada for encontrada, mova-a
    if linha_escolhida_definida:
      instrucoes.insert(hazards[i] + 1, linha_escolhida)
      instrucoes.remove(instrucoes[indice_linha_escolhida + 1])

  return instrucoes


In [ ]:
def verificar_hazard_instrucao(instrucao_origem, instrucao_destino, forwarding_implementado):
  """
  Verifica se há hazard entre duas instruções de assembly.

  Argumentos:
    instrucao_origem: Objeto LinhaASM representando a instrução de origem.
    instrucao_destino: Objeto LinhaASM representando a instrução de destino.
    forwarding_implementado: Booleano indicando se o forwarding está implementado.

  Retorna:
    Booleano indicando se há hazard.
  """

  # Instruções tipo S não causam hazard
  if instrucao_origem.tipo_instrucao == "S":
    return False

  # Ignorar NOPs manuais e ecalls
  if instrucao_origem.is_manual_nop or instrucao_destino.is_manual_nop or instrucao_destino.instrucao == "00000000000000000000000001110011":
    return False

  # Se o encaminhamento está sendo implementado, verifica apenas hazard de lw
  if forwarding_implementado and instrucao_origem.tipo_instrucao != "I_lo":
    return False

  # Verifica hazards para instruções R, I_ar, I_lo, S e B
  if instrucao_destino.tipo_instrucao in ["R", "I_ar", "I_lo", "S", "B"]:
    if instrucao_origem.rd == instrucao_destino.rs1:
      print("RD de origem conflita com rs1!")
      return True

  if instrucao_destino.tipo_instrucao in ["R", "S", "B"]:
    if instrucao_origem.rd == instrucao_destino.rs2:
      print("RD de origem conflita com rs2!")
      return True

  return False

In [ ]:
def inserir_nops(instrucoes, hazards, forwarding_implementado):
  """
  Insere instruções NOP (No Operation) em um vetor de instruções de assembly.

  Argumentos:
    instrucoes: Vetor de objetos LinhaASM representando as instruções de assembly.
    hazards: Vetor de índices de instruções que possuem hazards.
    forwarding_implementado: Booleano indicando se o forwarding está implementado.

  Retorna:
    Vetor de instruções de assembly modificado com os NOPs inseridos.
  """

  no_operator = LinhaASM()  # Objeto LinhaASM para representar a instrução NOP
  no_operator.instrucao = "00000000000000000000000000110011"  # Binário da instrução NOP
  no_operator.opcode = no_operator.instrucao[25:32]  # Extrai o opcode
  no_operator.rd = no_operator.instrucao[20:25]  # Extrai o registrador de destino
  no_operator.funct3 = no_operator.instrucao[17:20]  # Extrai o funct3
  no_operator.rs1 = no_operator.instrucao[12:17]  # Extrai o registrador de origem 1
  no_operator.rs2 = no_operator.instrucao[7:12]  # Extrai o registrador de origem 2
  no_operator.funct7 = no_operator.instrucao[0:7]  # Extrai o funct7
  no_operator.tipo_instrucao = ler_opcode(no_operator.opcode)  # Determina o tipo de instrução
  no_operator.is_manual_nop = True  # Indica que é um NOP manual

  # Itera pelos hazards na ordem inversa
  for i in range(len(hazards) - 1, -1, -1):
    # Determina a quantidade de NOPs a serem inseridos
    quant_nops = 2 if not forwarding_implementado else 1

    # Itera pelas instruções que podem ter hazards com o hazard atual
    for j in range(hazards[i] + 1, hazards[i] + quant_nops + 1):
      # Ignora se a iteração ultrapassa o limite de instruções
      if j >= len(instrucoes):
        continue

      # Verifica se há hazard entre a instrução atual e a instrução de destino do hazard
      if verificar_hazard_instrucao(instrucoes[hazards[i]], instrucoes[j], forwarding_implementado):
        # Insere os NOPs necessários
        for _ in range(quant_nops):
          #OTO_DANDO_ERRO
          instrucoes.insert(i + 1, no_operator)

        # Declina a quantidade de NOPs a serem inseridos
        quant_nops -= 1

    # Exibe mensagem de sucesso
    print("NO OPERATORS INSERIDOS COM SUCESSO")

    # Retorna as instruções modificadas
    return inserir_nops_em_jump(instrucoes)

In [ ]:
def ler_arquivo(arquivo):
  """
  Lê instruções assembly de um arquivo e retorna uma lista de objetos LinhaASM.

  Argumento:
    arquivo: Um objeto de arquivo aberto.

  Retorna:
    Uma lista de objetos LinhaASM representando as instruções de montagem.
  """
  delimiter = '\n'
  line_number = 0
  retorno = []
  with open(arquivo, 'r') as f:
      for line in f:
        line_number += 1
        linha_atual = LinhaASM()
        linha_atual.instrucao =  line.strip(delimiter)
        if len(linha_atual.instrucao) != 32:
           raise RuntimeError(f"Não foi possivel pois a linha: {line_number} não possui 32 digitos, cheque o arquivo.")

        linha_atual.opcode = linha_atual.instrucao[25:32]
        linha_atual.rd = linha_atual.instrucao[20:25]
        linha_atual.funct3 = linha_atual.instrucao[17:20]
        linha_atual.rs1 = linha_atual.instrucao[12:17]
        linha_atual.rs2 = linha_atual.instrucao[7:12]
        linha_atual.funct7 = linha_atual.instrucao[0:7]
        # Determine o tipo de instrução no opcode
        linha_atual.tipo_instrucao = ler_opcode(linha_atual.opcode)

        retorno.append(linha_atual)

  return retorno


In [ ]:
def ler_opcode(opcode):
  """
  Determina o tipo de instrução com base no opcode.

  Argumento:
    opcode: Uma string que representa o opcode.

  Retorna:
    Uma string que representa o tipo de instrução (por exemplo, "U", "J", "I_ar", etc.).
  """

  opcode_map = {
      "0110111": "U",
      "0010111": "U",
      "1101111": "J",
      "1100011": "B",
      "1100111": "I_ar",
      "0010011": "I_ar",
      "0001111": "I_ar",
      "1110011": "I_ar",
      "0000011": "I_lo",
      "0110011": "R",
      "0100011": "S"
  }

  if opcode in opcode_map:
    return opcode_map[opcode]
  else:
    print(f"(Aviso) Opcode desconhecido: {opcode}")
    return "?"


In [ ]:
def inserir_nops_em_jump(instrucoes):
    """
   Insere uma instrução NOP antes de cada instrução de salto.

    Argumento:
      instrucoes: Uma lista de objetos LinhaASM que representam instruções de montagem.

    Retorna:
      Uma lista de objetos LinhaASM com NOPs inseridos antes dos saltos.
    """

    no_operator = LinhaASM()  # Objeto representando instrução NOP
    no_operator.instrucao = "00000000000000000000000000110011"
    no_operator.is_manual_nop = True

    # Iterar pelas instruções na ordem inversa
    for i in range(len(instrucoes) - 1, -1, -1):
      # Verifique se a instrução atual é um salto
      if instrucoes[i].tipo_instrucao == "J":
        # Insira NOP antes da instrução de salto
        instrucoes.insert(i, no_operator)

    return instrucoes


NÃO CONFUNTA 'R' COM 'W' <br>
NÃO É RWITE É WRITE  <br>
SEMANAS ATRAS DE COMO QUEBRAR LINHA É COMO HTML

In [ ]:
def salvar_programa(programa, nome_programa):
  """
Salva as instruções de montagem modificadas em um arquivo.

  Argumentos:
    programa: Uma lista de objetos LinhaASM que representam as instruções modificadas.
    nome_programa: O nome do arquivo de saída.
  """

  # Abra o arquivo de saída no modo de gravação
  with open(nome_programa, "w") as arquivo_final:
    # Check if the file is open

    # Escreva cada instrução no arquivo
    for linha in programa:
      arquivo_final.write(f"{linha.instrucao}\n")

  # Imprima uma mensagem de sucesso
  print(f"Instruções salvas com sucesso em {nome_programa}")

In [ ]:
def VisualizarInstrucoes(programa, destacar_nops=True):
  """
 Imprime as instruções de montagem com destaque NOP opcional.

  Argumentos:
    programa: Uma lista de objetos LinhaASM que representam as instruções.
    destacar_nops: Um booleano que indica se NOPs devem ser destacados (padrão: True).
  """

  print("----------------------")

  for i, instrucao in enumerate(programa, start=1):
    print(f"{i}: {instrucao.instrucao}", end="")
    if destacar_nops and instrucao.is_manual_nop:
      print(" | NOP |", end="")
    print("\n")

  print("----------------------")

Inserir o path do arquivo aqui

In [ ]:
def solucao(tecnica):
  """
  Generates a solution to the hazard problem using the specified technique.

  Arguments:
    tecnica: The technique to use (1, 2, 3, or 4).
    nome_fornecido: The name of the input assembly file.

  Returns:
    A list of LinhaASM objects representing the modified instructions.
  """
  print(f"\nExecuting solution {tecnica}")


  path_to_file = "/content/test.txt"

  if path_to_file == "batatinha123":
    print(f"COLOQUE O CAMINHO/DO/ARQUIVO/NOMEDOARQUIVO.TXT")
  else :
    # Read the assembly instructions
    instrucoes = ler_arquivo(path_to_file)

    # Display the original instructions
    print("Original instructions:")
    VisualizarInstrucoes(instrucoes)

    # Initialize variables
    falhas = []
    forwarding_implementado = False

    # Apply the selected technique
    if tecnica == 1:
      # Technique 1: NOPs M2A
      forwarding_implementado = False
      falhas = verificar_hazards(instrucoes, forwarding_implementado)
      instrucoes = inserir_nops(instrucoes, falhas, forwarding_implementado)
      salvar_programa(instrucoes, "Solucao_1_NOP-SEM-Forward.txt")

    elif tecnica == 2:
      # Technique 2: NOPs with forwarding
      forwarding_implementado = True
      falhas = verificar_hazards(instrucoes, forwarding_implementado)
      instrucoes = inserir_nops(instrucoes, falhas, forwarding_implementado)
      salvar_programa(instrucoes, "Solucao_2_NOP-Forward.txt")

    elif tecnica == 3:
      # Technique 3: Reordering, NOPs M2A
      forwarding_implementado = False
      falhas = verificar_hazards(instrucoes, forwarding_implementado)
      instrucoes = aplicar_reordenacao(instrucoes, falhas, forwarding_implementado)
      falhas = verificar_hazards(instrucoes, forwarding_implementado)
      instrucoes = inserir_nops(instrucoes, falhas, forwarding_implementado)
      salvar_programa(instrucoes, "Solucao_3_Reordenamento-NOP-SEM-Forward.txt")

    elif tecnica == 4:
      # Technique 4: Reordering, NOPs with forwarding
      forwarding_implementado = True
      falhas = verificar_hazards(instrucoes, forwarding_implementado)
      instrucoes = aplicar_reordenacao(instrucoes, falhas, forwarding_implementado)
      falhas = verificar_hazards(instrucoes, forwarding_implementado)
      instrucoes = inserir_nops(instrucoes, falhas, forwarding_implementado)
      salvar_programa(instrucoes, "Solucao_4_Reordenamento-NOP-Forward.txt")

    else:
      print(f"[WARNING] Solution {tecnica} not implemented!")
      return []

  return instrucoes

In [ ]:
def calcular_resultados(programa, organizacao):
    resultado = Resultados()

    for i in range(len(programa)):
        resultado.ciclos_totais += 5 if i == 0 else 1

    print(f"\nTotal de ciclos: {resultado.ciclos_totais}\n")
    resultado.cpi = gerar_cpi(resultado.ciclos_totais, len(programa))
    resultado.tempo_exec = t_exec_cpu_por_tempo_clock(len(programa), resultado.cpi, organizacao.TClock)
    resultado.desempenho = gerar_desempenho(resultado.tempo_exec)

    return resultado

Main

Altere o choice para o tipo de solução:

1. Inserção de NOPS

2.  Reordenação e inserção de NOPS

3. Forwarding e inserção de NOPS

4. Forwarding, reordenação e inserção de NOPS

<strong> Se der erro aqui altere o path do arquivo </strong>

In [ ]:
clear_output(wait=True)

org_a = Organizacao()
saida = []

choice = 4 # escolha qual o problema a resolver

if choice < 1 or choice > 4:
  print(f"Algúm numero que esteja na lista é necessário pra funcionar!!!")
else:
  try:
    saida = solucao(choice)
    resultado = Resultados(calcular_resultados(saida, org_a))
    print("\n")
    print(f"CPI (Ciclos por Instrucao): {resultado.cpi}\n")
    print(f"Tempo de execucao: {resultado.tempo_exec}\n")
    print(f"Desempenho: {resultado.desempenho}\n")
  except RuntimeError as e:
    clear_output(wait=True)
    print(f"Erro encontrado:\n")
    print(e)






Executing solution 4
Original instructions:
----------------------
1: 00000100010000000000000001101111

2: 00000010101111111101111001100011

3: 00000000000000000000001010110011

4: 00000010101100101101011001100011

5: 00000000001000101001111010010011

6: 00000001110101010000111100110011

7: 00000000000011110010001110000011

8: 00000000010011110010111000000011

9: 00000000011111100100010001100011

10: 00000000110000000000000001101111

11: 00000001110011110010000000100011

12: 00000000011111110010001000100011

13: 00000000000100101000001010010011

14: 11111101100111111111000001101111

15: 00000000000111111000111110010011

16: 11111100100111111111000001101111

17: 00000000000000001000000011100111

18: 00001111110000010000010100010111

19: 11111011110001010000010100010011

20: 00000000011100000000010110010011

21: 11111011010111111111000011101111

22: 00000000101000000000100010010011

23: 00000000000000000000000001110011

----------------------
Executando verificação de hazards
RD de orig